# Warsztat 2: Ingestion Pipeline - COPY INTO & Auto Loader

**Cel warsztatu:**
- Implementacja batch ingestion używając COPY INTO
- Konfiguracja Auto Loader dla streaming ingestion
- Obsługa różnych formatów plików (CSV, JSON, Parquet)
- Monitorowanie i zarządzanie pipeline'ami ingestion

**Czas:** 90 minut

---

## 📚 Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

## 🎯 Część 1: COPY INTO - Batch Ingestion

### Zadanie 1.1: Ingestion plików CSV

**Instrukcje:**
1. Przygotuj tabelę docelową `bronze_customers_batch`
2. Użyj `COPY INTO` do załadowania danych z `customers.csv`
3. Zweryfikuj liczbę załadowanych rekordów

In [ ]:
# TODO: Utwórz tabelę docelową
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_customers_batch (
        customer_id INT,
        name STRING,
        email STRING,
        city STRING,
        country STRING,
        _ingestion_timestamp TIMESTAMP
    )
    USING DELTA
    LOCATION '{____}/customers_batch'  -- Uzupełnij BRONZE_PATH
""")

In [ ]:
# TODO: COPY INTO z pliku CSV
spark.sql(f"""
    ____ INTO {CATALOG}.{SCHEMA}.bronze_customers_batch
    FROM (
        SELECT 
            customer_id,
            name,
            email,
            city,
            country,
            current_timestamp() as _ingestion_timestamp
        FROM '{____}/____'  -- Uzupełnij SOURCE_DATA_PATH i nazwę pliku
    )
    FILEFORMAT = ____  -- Uzupełnij format (CSV)
    FORMAT_OPTIONS (
        'header' = '____',  -- Czy plik ma nagłówek?
        'inferSchema' = '____'  -- Czy inferować schemat?
    )
    COPY_OPTIONS (
        'mergeSchema' = '____'  -- Czy mergować schemat?
    )
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
    SELECT COUNT(*) as total_records 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").show()

### Zadanie 1.2: Ingestion plików JSON

**Instrukcje:**
1. Przygotuj tabelę `bronze_orders_batch`
2. Użyj `COPY INTO` do załadowania danych z `orders_batch.json`
3. Obsłuż zagnieżdżoną strukturę JSON

In [ ]:
# TODO: Utwórz tabelę dla zamówień
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_orders_batch (
        order_id INT,
        customer_id INT,
        order_date DATE,
        total_amount DOUBLE,
        status STRING,
        _ingestion_timestamp TIMESTAMP
    )
    USING ____  -- Uzupełnij format (DELTA)
    LOCATION '{____}/orders_batch'  -- Uzupełnij ścieżkę
""")

In [ ]:
# TODO: COPY INTO z pliku JSON
spark.sql(f"""
    COPY INTO {CATALOG}.{SCHEMA}.bronze_orders_batch
    FROM (
        SELECT 
            order_id,
            customer_id,
            TO_DATE(order_date) as order_date,
            total_amount,
            status,
            current_timestamp() as _ingestion_timestamp
        FROM '{____}/____'  -- Uzupełnij ścieżkę do JSON
    )
    FILEFORMAT = ____  -- Uzupełnij format
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
    SELECT * FROM {CATALOG}.{SCHEMA}.bronze_orders_batch LIMIT 10
""").show()

### Zadanie 1.3: Ingestion plików Parquet

**Instrukcje:**
1. Przygotuj tabelę `bronze_products_batch`
2. Użyj `COPY INTO` do załadowania danych z `products.parquet`
3. Dodaj kolumnę z metadanymi pliku źródłowego

In [ ]:
# TODO: Utwórz tabelę dla produktów
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {CATALOG}.{SCHEMA}.bronze_products_batch (
        product_id INT,
        product_name STRING,
        category STRING,
        price DOUBLE,
        stock_quantity INT,
        _source_file STRING,
        _ingestion_timestamp TIMESTAMP
    )
    USING DELTA
    LOCATION '{____}/products_batch'
""")

In [ ]:
# TODO: COPY INTO z pliku Parquet
spark.sql(f"""
    COPY INTO {CATALOG}.{SCHEMA}.bronze_products_batch
    FROM (
        SELECT 
            product_id,
            product_name,
            category,
            price,
            stock_quantity,
            ____ as _source_file,  -- Użyj _metadata.file_path
            current_timestamp() as _ingestion_timestamp
        FROM '{____}/____'  -- Uzupełnij ścieżkę do Parquet
    )
    FILEFORMAT = ____
""")

In [ ]:
# Weryfikacja
spark.sql(f"""
    SELECT product_id, product_name, category, _source_file 
    FROM {CATALOG}.{SCHEMA}.bronze_products_batch 
    LIMIT 10
""").show(truncate=False)

### Zadanie 1.4: Idempotentność - Ponowne uruchomienie COPY INTO

**Instrukcje:**
1. Uruchom ponownie `COPY INTO` dla tej samej tabeli
2. Zweryfikuj że dane nie zostały zduplikowane
3. Sprawdź historię operacji `COPY INTO`

In [ ]:
# Sprawdź liczbę rekordów przed ponownym COPY INTO
before_count = spark.sql(f"""
    SELECT COUNT(*) as count 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").collect()[0]["count"]

print(f"Liczba rekordów przed ponownym COPY INTO: {before_count}")

In [ ]:
# TODO: Ponowne wykonanie COPY INTO
spark.sql(f"""
    COPY INTO {CATALOG}.{SCHEMA}.bronze_customers_batch
    FROM (
        SELECT 
            customer_id,
            name,
            email,
            city,
            country,
            current_timestamp() as _ingestion_timestamp
        FROM '{SOURCE_DATA_PATH}/customers.csv'
    )
    FILEFORMAT = CSV
    FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
""")

In [ ]:
# Weryfikacja idempotentności
after_count = spark.sql(f"""
    SELECT COUNT(*) as count 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""").collect()[0]["count"]

print(f"Liczba rekordów po ponownym COPY INTO: {after_count}")
print(f"Czy dane zostały zduplikowane? {before_count != after_count}")

---

## 🔄 Część 2: Auto Loader - Streaming Ingestion

### Zadanie 2.1: Konfiguracja Auto Loader dla CSV

**Instrukcje:**
1. Przygotuj lokalizację checkpointu
2. Użyj `.format("cloudFiles")` do utworzenia streaming read
3. Skonfiguruj schema inference i evolution
4. Zapisz stream do tabeli `bronze_customers_stream`

In [ ]:
# TODO: Streaming read z Auto Loader
customers_stream = (
    spark.readStream
    .format("____")  # Uzupełnij format (cloudFiles)
    .option("cloudFiles.format", "____")  # Format plików źródłowych (csv)
    .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/____")  # Checkpoint dla schematu
    .option("header", "true")
    .load(f"{SOURCE_DATA_PATH}/customers.csv")
)

In [ ]:
# TODO: Dodaj kolumny metadata
from pyspark.sql.functions import current_timestamp, input_file_name

customers_enriched = (
    customers_stream
    .withColumn("_ingestion_timestamp", ____)  # Dodaj timestamp
    .withColumn("_source_file", ____)  # Dodaj nazwę pliku źródłowego
)

In [ ]:
# TODO: Zapisz stream do tabeli Delta
query_customers = (
    customers_enriched.writeStream
    .format("____")  # Uzupełnij format
    .outputMode("____")  # Tryb zapisu (append)
    .option("checkpointLocation", f"{____}/customers_stream")  # Checkpoint
    .option("mergeSchema", "____")  # Schema evolution
    .table(f"{CATALOG}.{SCHEMA}.bronze_customers_stream")
)

In [ ]:
# Weryfikacja streamu
import time
time.sleep(10)  # Poczekaj na przetworzenie

spark.sql(f"""
    SELECT COUNT(*) as total_records 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_stream
""").show()

### Zadanie 2.2: Auto Loader dla JSON z Schema Hints

**Instrukcje:**
1. Użyj Auto Loader do odczytu `orders_batch.json`
2. Dodaj schema hints dla kolumn z konkretnymi typami
3. Skonfiguruj rescue data column dla niepoprawnych rekordów

In [ ]:
# TODO: Auto Loader z schema hints
orders_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "____")  # Format JSON
    .option("cloudFiles.schemaLocation", f"{CHECKPOINT_PATH}/orders_schema")
    .option("cloudFiles.schemaHints", "____")  # Podpowiedź: "order_date DATE, total_amount DOUBLE"
    .option("cloudFiles.rescuedDataColumn", "____")  # Kolumna dla rescue data
    .load(f"{____}/orders_batch.json")  # Uzupełnij ścieżkę
)

In [ ]:
# TODO: Zapisz stream
query_orders = (
    orders_stream.writeStream
    .format("delta")
    .outputMode("____")  # Tryb append
    .option("checkpointLocation", f"{CHECKPOINT_PATH}/____")  # Checkpoint
    .table(f"{CATALOG}.{SCHEMA}.bronze_orders_stream")
)

In [ ]:
# Weryfikacja
time.sleep(10)
spark.sql(f"""
    SELECT * FROM {CATALOG}.{SCHEMA}.bronze_orders_stream LIMIT 10
""").show()

### Zadanie 2.3: Monitoring streaming queries

**Instrukcje:**
1. Wyświetl aktywne streaming queries
2. Sprawdź status i ostatni progress każdego query
3. Pobierz metryki: liczba przetworzonych rekordów, batch duration

In [ ]:
# TODO: Wyświetl aktywne streamy
active_streams = spark.streams.____  # Uzupełnij metodę (active)

print(f"Liczba aktywnych streamów: {len(active_streams)}")
for stream in active_streams:
    print(f"\nStream ID: {stream.id}")
    print(f"Name: {stream.name}")
    print(f"Status: {stream.status}")

In [ ]:
# TODO: Sprawdź ostatni progress
if len(active_streams) > 0:
    last_progress = active_streams[0].____  # Uzupełnij metodę (lastProgress)
    
    if last_progress:
        print(f"Batch ID: {last_progress['batchId']}")
        print(f"Przetworzone rekordy: {last_progress['numInputRows']}")
        print(f"Czas przetwarzania: {last_progress['batchDuration']} ms")

### Zadanie 2.4: Zatrzymanie streaming queries

**Instrukcje:**
1. Zatrzymaj wszystkie aktywne streaming queries
2. Zweryfikuj że wszystkie streamy są zatrzymane

In [ ]:
# TODO: Zatrzymaj wszystkie streamy
for stream in spark.streams.active:
    print(f"Zatrzymuję stream: {stream.name}")
    stream.____()  # Uzupełnij metodę (stop)

print("\nWszystkie streamy zatrzymane!")

In [ ]:
# Weryfikacja
print(f"Liczba aktywnych streamów: {len(spark.streams.active)}")

---

## 📊 Część 3: Porównanie COPY INTO vs Auto Loader

### Zadanie 3.1: Analiza wydajności

**Instrukcje:**
1. Porównaj liczbę rekordów załadowanych przez COPY INTO vs Auto Loader
2. Sprawdź historię operacji dla obu metod
3. Zidentyfikuj przypadki użycia dla każdej metody

In [ ]:
# Porównanie liczby rekordów
copy_into_count = spark.sql(f"""
    SELECT 'COPY INTO' as method, COUNT(*) as records 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_batch
""")

auto_loader_count = spark.sql(f"""
    SELECT 'Auto Loader' as method, COUNT(*) as records 
    FROM {CATALOG}.{SCHEMA}.bronze_customers_stream
""")

copy_into_count.union(auto_loader_count).show()

In [ ]:
# TODO: Historia operacji COPY INTO
spark.sql(f"""
    ____ HISTORY {CATALOG}.{SCHEMA}.bronze_customers_batch
""").select("version", "operation", "operationMetrics").show(truncate=False)

In [ ]:
# TODO: Historia operacji Auto Loader
spark.sql(f"""
    DESCRIBE HISTORY {CATALOG}.{SCHEMA}.bronze_customers_stream
""").select("version", "operation", "operationMetrics").show(truncate=False)

---

## ✅ Podsumowanie warsztatu

**Zrealizowane cele:**
- ✅ Implementacja batch ingestion z COPY INTO
- ✅ Konfiguracja Auto Loader dla streaming ingestion
- ✅ Obsługa różnych formatów (CSV, JSON, Parquet)
- ✅ Monitoring i zarządzanie pipeline'ami

**Kiedy użyć COPY INTO:**
- Batch processing z określonym harmonogramem
- Znana i stabilna struktura danych
- Potrzeba kontroli nad procesem ładowania
- Idempotentność out-of-the-box

**Kiedy użyć Auto Loader:**
- Near real-time processing
- Schema evolution i automatyczna inferencja
- Ciągłe monitorowanie nowych plików
- Skalowalność i efektywność kosztowa

---

## 🧹 Cleanup (opcjonalnie)

In [ ]:
# Usuń utworzone tabele (opcjonalnie)
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.bronze_customers_batch")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.bronze_orders_batch")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.bronze_products_batch")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.bronze_customers_stream")
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{SCHEMA}.bronze_orders_stream")

In [ ]:
```xml
<VSCode.Cell language="markdown">
# Ingestion Pipeline - Workshop

**Cel szkoleniowy:** Praktyczne zastosowanie różnych metod ładowania danych: COPY INTO (batch), Auto Loader (incremental), i Structured Streaming.

**Zakres tematyczny:**
- COPY INTO - batch file ingestion
- Auto Loader (CloudFiles) - incremental file ingestion
- Schema inference i schema evolution
- Structured Streaming - real-time processing
- Checkpointing i exactly-once semantics
- MERGE na streamingu

**Czas trwania:** 90 minut
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:
</VSCode.Cell>
<VSCode.Cell language="python">
%run ../../00_setup
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:
</VSCode.Cell>
<VSCode.Cell language="python">
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Ścieżki do danych
BASE_DATA_PATH = f"{DATASET_BASE_PATH}"
ORDERS_PATH = f"{BASE_DATA_PATH}/orders"
CUSTOMERS_PATH = f"{BASE_DATA_PATH}/customers"

# Ścieżka do checkpointów (per user)
CHECKPOINT_BASE = f"/tmp/{raw_user}/checkpoints"

print(f"\n=== Ścieżki ===")
print(f"Data path: {BASE_DATA_PATH}")
print(f"Checkpoint path: {CHECKPOINT_BASE}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 1: COPY INTO - Batch File Ingestion (20 minut)

**Cel:** Użycie COPY INTO do efektywnego ładowania plików batch'owych z automatyczną deduplikacją.

### Zadanie 1.1: COPY INTO dla plików JSON

**Instrukcje:**
1. Utwórz tabelę docelową `orders_copy_into` jeśli nie istnieje
2. Użyj `COPY INTO` do załadowania plików JSON z folderu `orders/`
3. Parametry:
   - FORMAT = JSON
   - PATTERN = '*.json'
   - Dodaj audit columns: `_metadata.file_path`, `_metadata.file_modification_time`

**Oczekiwany rezultat:**
- Dane załadowane z automatyczną deduplikacją (COPY INTO śledzi załadowane pliki)

**Wskazówki:**
- COPY INTO automatycznie pomija już załadowane pliki (idempotentność)
- Używaj `_metadata` dla audit trail
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.1 - COPY INTO dla JSON

# Nazwa tabeli docelowej
copy_into_table = f"{BRONZE_SCHEMA}.orders_copy_into"

# Utwórz pustą tabelę jeśli nie istnieje
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {copy_into_table} (
        order_id BIGINT,
        customer_id BIGINT,
        order_date STRING,
        order_amount DOUBLE,
        order_status STRING,
        source_file STRING,
        load_timestamp TIMESTAMP
    ) USING DELTA
""")

print(f"✓ Tabela utworzona: {copy_into_table}")

# Sprawdź stan przed COPY INTO
count_before = spark.table(copy_into_table).count()
print(f"Liczba rekordów przed COPY INTO: {count_before}")

# COPY INTO - załaduj pliki JSON
spark.sql(f"""
    COPY INTO {copy_into_table}
    FROM (
        SELECT 
            order_id,
            customer_id,
            order_date,
            order_amount,
            order_status,
            _metadata.____ AS source_file,
            current_timestamp() AS load_timestamp
        FROM '{____}'
    )
    FILEFORMAT = ____
    PATTERN = '____'
    FORMAT_OPTIONS ('multiLine' = 'true')
""")

# Sprawdź stan po COPY INTO
count_after = spark.table(copy_into_table).count()
print(f"\n=== Wynik COPY INTO ===")
print(f"Liczba rekordów po: {count_after}")
print(f"Nowo załadowane: {count_after - count_before}")

# Wyświetl dane
display(spark.table(copy_into_table).limit(5))

print("✓ COPY INTO zakończony!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 1.2: COPY INTO - Idempotentność (ponowne uruchomienie)

**Instrukcje:**
1. Uruchom ponownie COPY INTO z poprzedniego zadania
2. Sprawdź, czy liczba rekordów się nie zmieniła (idempotentność)

**Oczekiwany rezultat:**
- Liczba rekordów taka sama - COPY INTO pomija już załadowane pliki
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.2 - Test idempotentności COPY INTO

# Liczba przed ponownym COPY INTO
count_before_rerun = spark.table(copy_into_table).count()
print(f"Liczba rekordów przed ponownym uruchomieniem: {count_before_rerun}")

# Uruchom COPY INTO ponownie (skopiuj kod z Zadania 1.1)
spark.sql(f"""
    ____ ____ {copy_into_table}
    FROM (
        SELECT 
            order_id,
            customer_id,
            order_date,
            order_amount,
            order_status,
            _metadata.file_path AS source_file,
            current_timestamp() AS load_timestamp
        FROM '{ORDERS_PATH}'
    )
    FILEFORMAT = JSON
    PATTERN = '*.json'
    FORMAT_OPTIONS ('multiLine' = 'true')
""")

# Liczba po ponownym COPY INTO
count_after_rerun = spark.table(copy_into_table).count()
print(f"\n=== Test idempotentności ===")
print(f"Liczba rekordów po ponownym uruchomieniu: {count_after_rerun}")
print(f"Różnica: {count_after_rerun - count_before_rerun}")

if count_after_rerun == count_before_rerun:
    print("✓ COPY INTO jest idempotentny - pliki nie zostały załadowane ponownie!")
else:
    print("⚠ Uwaga: wykryto duplikaty - sprawdź konfigurację")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 2: Auto Loader (CloudFiles) - Incremental Ingestion (25 minut)

**Cel:** Użycie Auto Loader do automatycznego ładowania nowych plików z schema inference i evolution.

### Zadanie 2.1: Auto Loader - podstawowa konfiguracja

**Instrukcje:**
1. Użyj `spark.readStream.format("cloudFiles")` do wczytywania CSV z `customers/`
2. Włącz schema inference: `.option("cloudFiles.schemaLocation", checkpoint_path)`
3. Dodaj audit column: `_metadata.file_path`
4. Zapisz do tabeli `customers_autoloader` używając `writeStream`
5. Trigger: `availableNow=True` (process all available, then stop)

**Oczekiwany rezultat:**
- Auto Loader automatycznie wykrywa nowe pliki
- Schema inference działa automatycznie

**Wskazówki:**
- Auto Loader wymaga checkpoint location dla tracking files
- Schema location przechowuje inferred schema
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.1 - Auto Loader basics

# Ścieżki
autoloader_table = f"{BRONZE_SCHEMA}.customers_autoloader"
checkpoint_path = f"{CHECKPOINT_BASE}/customers_autoloader"
schema_path = f"{CHECKPOINT_BASE}/customers_schema"

print(f"=== Konfiguracja Auto Loader ===")
print(f"Tabela: {autoloader_table}")
print(f"Checkpoint: {checkpoint_path}")
print(f"Schema location: {schema_path}")

# Auto Loader - read stream
customers_stream = (
    spark.readStream
    .format("____")  # cloudFiles
    .option("cloudFiles.format", "____")  # csv
    .option("header", "true")
    .option("cloudFiles.schemaLocation", ____)  # schema_path
    .load(____)  # CUSTOMERS_PATH
    .withColumn("source_file", F.col("_metadata.____"))
    .withColumn("ingest_timestamp", F.current_timestamp())
)

print("\n=== Inferred Schema ===")
customers_stream.printSchema()

# Write stream do tabeli Delta
query = (
    customers_stream
    .writeStream
    .format("____")  # delta
    .option("checkpointLocation", ____)  # checkpoint_path
    .trigger(____=True)  # availableNow
    .table(____)  # autoloader_table
)

# Czekaj na zakończenie
query.awaitTermination()

# Sprawdź wyniki
count = spark.table(autoloader_table).count()
print(f"\n✓ Auto Loader zakończony!")
print(f"Załadowano rekordów: {count}")

display(spark.table(autoloader_table).limit(5))
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 2.2: Auto Loader - Schema Evolution

**Instrukcje:**
1. Symuluj dodanie nowej kolumny do źródłowych danych
2. Włącz schema evolution: `.option("cloudFiles.schemaEvolutionMode", "addNewColumns")`
3. Uruchom Auto Loader ponownie
4. Sprawdź czy nowa kolumna została dodana automatycznie

**Oczekiwany rezultat:**
- Schema evolution automatycznie dodaje nową kolumnę

**Wskazówki:**
- Schema evolution modes: `none`, `addNewColumns`, `rescue`
- Rescue mode zapisuje niepasujące dane do `_rescued_data` column
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.2 - Schema Evolution

# Symulacja: dodaj nową kolumnę do danych (w praktyce nowy plik w cloud storage)
# Dla demonstracji użyjemy istniejących danych + nową kolumnę

print("=== Test Schema Evolution ===")
print("Symulacja: nowy plik CSV z dodatkową kolumną 'loyalty_tier'")

# Utwórz testowy DataFrame z nową kolumną
test_data = spark.table(autoloader_table).limit(10).withColumn("loyalty_tier", F.lit("Gold"))

# Zapisz jako tymczasowy CSV (symulacja nowego pliku)
temp_csv_path = f"/tmp/{raw_user}/test_customers_new_column"
test_data.write.format("csv").mode("overwrite").option("header", "true").save(temp_csv_path)

print(f"✓ Utworzono testowy plik z nową kolumną: {temp_csv_path}")

# Nowa ścieżka checkpoint dla evolution test
evolution_checkpoint = f"{CHECKPOINT_BASE}/customers_evolution"
evolution_schema = f"{CHECKPOINT_BASE}/customers_evolution_schema"

# Auto Loader z schema evolution
customers_evolved_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.schemaLocation", evolution_schema)
    .option("cloudFiles.schemaEvolutionMode", "____")  # addNewColumns
    .load(temp_csv_path)
    .withColumn("source_file", F.col("_metadata.file_path"))
    .withColumn("ingest_timestamp", F.current_timestamp())
)

print("\n=== Evolved Schema (z nową kolumną) ===")
customers_evolved_stream.printSchema()

# Write stream
evolution_table = f"{BRONZE_SCHEMA}.customers_evolution_test"

query_evolution = (
    customers_evolved_stream
    .writeStream
    .format("delta")
    .option("checkpointLocation", evolution_checkpoint)
    .option("mergeSchema", "____")  # true - włącz schema merge
    .trigger(availableNow=True)
    .table(evolution_table)
)

query_evolution.awaitTermination()

print(f"\n✓ Schema Evolution zakończony!")
print("Sprawdź schema - powinna być nowa kolumna 'loyalty_tier':")

spark.table(evolution_table).printSchema()
display(spark.table(evolution_table).select("customer_id", "email", "loyalty_tier").limit(5))
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 3: Structured Streaming - Real-time Processing (25 minut)

**Cel:** Implementacja streaming pipeline z continuous processing i MERGE INTO.

### Zadanie 3.1: Streaming read + aggregation

**Instrukcje:**
1. Użyj `readStream` do wczytywania z Delta table `orders_copy_into`
2. Agreguj dane w czasie rzeczywistym:
   - Grupuj po `order_status`
   - Policz liczbę zamówień per status
   - Zsumuj `order_amount` per status
3. Zapisz do tabeli `order_status_summary` jako streaming table
4. Trigger: `processingTime='10 seconds'`

**Oczekiwany rezultat:**
- Streaming aggregation aktualizuje summary w czasie rzeczywistym

**Wskazówki:**
- Streaming DataFrame używa incremental processing
- OutputMode dla agregacji: `complete` (pełna tabela) lub `update` (tylko zmiany)
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.1 - Streaming aggregation

# Tabela źródłowa (już załadowana w Zadaniu 1)
source_table = copy_into_table

# Ścieżka checkpoint
streaming_checkpoint = f"{CHECKPOINT_BASE}/streaming_aggregation"

# Read stream z Delta table
orders_stream = (
    spark.readStream
    .format("____")  # delta
    .table(____)  # source_table
)

# Agregacja: liczba zamówień i suma per status
order_summary_stream = (
    orders_stream
    .groupBy("____")  # order_status
    .agg(
        F.count("____").alias("____"),  # order_id, order_count
        F.sum("____").alias("____"),    # order_amount, total_amount
        F.avg("____").alias("____")     # order_amount, avg_amount
    )
    .withColumn("updated_at", F.current_timestamp())
)

print("=== Streaming Aggregation Schema ===")
order_summary_stream.printSchema()

# Write stream do tabeli summary
summary_table = f"{BRONZE_SCHEMA}.order_status_summary"

query_summary = (
    order_summary_stream
    .writeStream
    .format("delta")
    .outputMode("____")  # complete - pełna tabela przy każdym update
    .option("checkpointLocation", streaming_checkpoint)
    .trigger(processingTime="____ seconds")  # 10
    .table(summary_table)
)

# Uruchom przez 30 sekund, potem stop
import time
time.sleep(30)
query_summary.stop()

print(f"\n✓ Streaming zakończony!")
print(f"Wyniki agregacji:")

display(spark.table(summary_table))
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 3.2: Streaming MERGE (Upsert)

**Cel:** Użycie `foreachBatch` do wykonania MERGE w streaming pipeline.

**Instrukcje:**
1. Użyj streaming source z `customers_autoloader`
2. Zaimplementuj funkcję `upsert_to_delta` która wykonuje MERGE INTO
3. Użyj `.foreachBatch(upsert_to_delta)` w writeStream
4. MERGE logic:
   - MATCHED: update email, age
   - NOT MATCHED: insert new customer

**Oczekiwany rezultat:**
- Streaming pipeline wykonuje upsert (MERGE) dla każdego micro-batch

**Wskazówki:**
- `foreachBatch` pozwala na custom logic per batch
- Użyj `microBatchOutputDF` w funkcji upsert
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.2 - Streaming MERGE

# Tabela docelowa
target_merge_table = f"{BRONZE_SCHEMA}.customers_merged"

# Utwórz tabelę docelową jeśli nie istnieje
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {target_merge_table} (
        customer_id BIGINT,
        first_name STRING,
        last_name STRING,
        email STRING,
        age INT,
        country STRING,
        registration_date STRING,
        last_updated TIMESTAMP
    ) USING DELTA
""")

print(f"✓ Tabela docelowa: {target_merge_table}")

# Funkcja upsert - wykonywana dla każdego micro-batch
def upsert_to_delta(microBatchDF, batchId):
    """
    Upsert function dla streaming MERGE
    """
    print(f"\n=== Processing batch {batchId} ===")
    print(f"Batch size: {microBatchDF.count()} records")
    
    # Dodaj timestamp
    microBatchDF = microBatchDF.withColumn("last_updated", F.current_timestamp())
    
    # Zarejestruj jako temp view
    microBatchDF.createOrReplaceTempView("updates")
    
    # MERGE INTO
    spark.sql(f"""
        MERGE INTO {target_merge_table} AS target
        USING updates AS source
        ON target.____ = source.____
        WHEN ____ THEN
            UPDATE SET
                target.email = source.____,
                target.age = source.____,
                target.last_updated = source.____
        WHEN ____ ____ THEN
            INSERT *
    """)
    
    print(f"✓ Batch {batchId} merged!")

# Read stream z Auto Loader table
customers_stream_merge = (
    spark.readStream
    .format("delta")
    .table(autoloader_table)
)

# Checkpoint
merge_checkpoint = f"{CHECKPOINT_BASE}/streaming_merge"

# Write stream z foreachBatch
query_merge = (
    customers_stream_merge
    .writeStream
    .foreachBatch(____)  # upsert_to_delta
    .option("checkpointLocation", ____)  # merge_checkpoint
    .trigger(____=True)  # availableNow
    .start()
)

query_merge.awaitTermination()

print(f"\n✓ Streaming MERGE zakończony!")
print(f"Liczba rekordów w tabeli docelowej: {spark.table(target_merge_table).count()}")

display(spark.table(target_merge_table).limit(5))
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 4: Porównanie metod ingestion (10 minut)

**Cel:** Zrozumienie różnic między COPY INTO, Auto Loader i Streaming.

### Zadanie 4.1: Analiza porównawcza

**Instrukcje:**
1. Wypełnij tabelę porównawczą poniżej
2. Dla każdej metody określ:
   - Use case (kiedy używać)
   - Idempotency (czy automatyczna)
   - Schema evolution (czy wspierana)
   - Performance (batch vs streaming)

**Oczekiwany rezultat:**
- Kompletna tabela porównawcza
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.1 - Analiza porównawcza

# Wypełnij tabelę porównawczą

comparison_data = [
    {
        "Method": "COPY INTO",
        "Use Case": "____",  # np. "Batch loads, daily/hourly ingestion"
        "Idempotency": "____",  # "Automatic" / "Manual"
        "Schema Evolution": "____",  # "Manual" / "Automatic"
        "Performance": "____",  # "Fast for batch" / "Slower"
        "Complexity": "____"  # "Low" / "Medium" / "High"
    },
    {
        "Method": "Auto Loader",
        "Use Case": "____",  # np. "Incremental file loads, landing zones"
        "Idempotency": "____",
        "Schema Evolution": "____",
        "Performance": "____",
        "Complexity": "____"
    },
    {
        "Method": "Structured Streaming",
        "Use Case": "____",  # np. "Real-time, continuous processing"
        "Idempotency": "____",
        "Schema Evolution": "____",
        "Performance": "____",
        "Complexity": "____"
    }
]

comparison_df = spark.createDataFrame(comparison_data)

print("=== Porównanie metod ingestion ===")
display(comparison_df)

# Twoja analiza:
print("""
=== Kiedy używać której metody? ===

COPY INTO:
- ____

Auto Loader:
- ____

Structured Streaming:
- ____
""")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] COPY INTO: Tabela `orders_copy_into` załadowana z idempotentością
- [ ] Auto Loader: Tabela `customers_autoloader` z automatyczną schema inference
- [ ] Schema Evolution: Tabela `customers_evolution_test` z nową kolumną
- [ ] Streaming Aggregation: Tabela `order_status_summary` z real-time agregacjami
- [ ] Streaming MERGE: Tabela `customers_merged` z upsert logic
- [ ] Analiza porównawcza wypełniona

### Komendy weryfikacyjne:
</VSCode.Cell>
<VSCode.Cell language="python">
# Weryfikacja wyników

print("=== WERYFIKACJA WYNIKÓW ===\n")

# 1. COPY INTO
print("1. COPY INTO:")
copy_count = spark.table(copy_into_table).count()
print(f"   Liczba rekordów: {copy_count}")
assert copy_count > 0, "Błąd: Tabela pusta"

# 2. Auto Loader
print("\n2. Auto Loader:")
autoloader_count = spark.table(autoloader_table).count()
print(f"   Liczba rekordów: {autoloader_count}")
assert autoloader_count > 0, "Błąd: Tabela pusta"

# 3. Schema Evolution
print("\n3. Schema Evolution:")
evolution_columns = spark.table(evolution_table).columns
print(f"   Kolumny: {evolution_columns}")
assert "loyalty_tier" in evolution_columns, "Błąd: Brak nowej kolumny loyalty_tier"
print("   ✓ Nowa kolumna loyalty_tier wykryta!")

# 4. Streaming Aggregation
print("\n4. Streaming Aggregation:")
summary_count = spark.table(summary_table).count()
print(f"   Liczba statusów: {summary_count}")
display(spark.table(summary_table))

# 5. Streaming MERGE
print("\n5. Streaming MERGE:")
merge_count = spark.table(target_merge_table).count()
print(f"   Liczba rekordów: {merge_count}")

# 6. Checkpoints
print("\n6. Checkpoints (powinny istnieć):")
checkpoints = [
    checkpoint_path,
    streaming_checkpoint,
    merge_checkpoint
]
for cp in checkpoints:
    exists = "✓ Exists" if spark._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark._jsc.hadoopConfiguration()
    ).exists(spark._jvm.org.apache.hadoop.fs.Path(cp)) else "✗ Missing"
    print(f"   {cp}: {exists}")

print("\n✓ Wszystkie testy przeszły pomyślnie!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Troubleshooting

**Problem 1: Auto Loader - Schema conflicts**
**Objawy:** Błąd: "Schema mismatch" podczas Auto Loader

**Rozwiązanie:**
```python
# Usuń checkpoint i schema location, zacznij od nowa
dbutils.fs.rm(checkpoint_path, True)
dbutils.fs.rm(schema_path, True)

# Lub włącz schema evolution:
.option("cloudFiles.schemaEvolutionMode", "addNewColumns")
```

**Problem 2: Streaming - Checkpoint conflicts**
**Objawy:** Błąd: "Incompatible checkpoint"

**Rozwiązanie:**
```python
# Usuń checkpoint dla nowego streama
dbutils.fs.rm(checkpoint_path, True)

# Lub użyj nowej ścieżki checkpoint
checkpoint_path = f"{CHECKPOINT_BASE}/new_stream_{timestamp}"
```

**Problem 3: COPY INTO - No new files**
**Objawy:** COPY INTO nie ładuje danych ponownie

**Rozwiązanie:**
```python
# COPY INTO śledzi załadowane pliki (idempotentność)
# To jest FEATURE, nie bug!
# Jeśli chcesz re-load: użyj FORCE = TRUE

COPY INTO table_name
FROM 'path'
FILEFORMAT = JSON
COPY_OPTIONS ('force' = 'true')  # Re-load all files
```

**Problem 4: Streaming - Memory issues**
**Rozwiązanie:**
```python
# Ogranicz shuffle partitions dla małych streamów
spark.conf.set("spark.sql.shuffle.partitions", "8")

# Użyj trigger availableNow zamiast continuous
.trigger(availableNow=True)
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Podsumowanie

**W tym warsztacie nauczyłeś się:**

✅ **COPY INTO:**
- Batch file ingestion z automatyczną deduplikacją
- Idempotentność - pliki ładowane tylko raz
- PATTERN matching dla selective loading
- Audit trail z _metadata

✅ **Auto Loader (CloudFiles):**
- Incremental file ingestion z automatycznym file tracking
- Schema inference - automatyczne wykrywanie schema
- Schema evolution - automatyczne dodawanie nowych kolumn
- Production-ready dla landing zones

✅ **Structured Streaming:**
- Real-time processing z micro-batches
- Streaming aggregations (complete output mode)
- foreachBatch dla custom logic (MERGE)
- Checkpointing dla exactly-once semantics

**Porównanie metod:**

| Metoda | Use Case | Idempotency | Schema Evolution | Performance |
|--------|----------|-------------|------------------|-------------|
| COPY INTO | Daily/hourly batch | Automatic | Manual | Fast batch |
| Auto Loader | Incremental files | Automatic | Automatic | Efficient incremental |
| Streaming | Real-time | Manual (checkpoint) | Manual/Rescue | Low latency |

**Kluczowe wnioski:**
1. COPY INTO - best dla scheduled batch loads (daily, hourly)
2. Auto Loader - best dla landing zones z unpredictable file arrival
3. Structured Streaming - best dla low-latency, real-time processing
4. Wszystkie metody wspierają exactly-once semantics z checkpointing

**Następne kroki:**
- **Kolejny warsztat**: 03_end_to_end_bronze_silver_gold_workshop.ipynb
- **Materiały dodatkowe**: Delta Lake Streaming Guide, Auto Loader Best Practices
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Cleanup

Opcjonalnie: usuń utworzone tabele i checkpointy:
</VSCode.Cell>
<VSCode.Cell language="python">
# Opcjonalne czyszczenie zasobów
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# Usuń tabele
# spark.sql(f"DROP TABLE IF EXISTS {copy_into_table}")
# spark.sql(f"DROP TABLE IF EXISTS {autoloader_table}")
# spark.sql(f"DROP TABLE IF EXISTS {evolution_table}")
# spark.sql(f"DROP TABLE IF EXISTS {summary_table}")
# spark.sql(f"DROP TABLE IF EXISTS {target_merge_table}")

# Usuń checkpointy
# dbutils.fs.rm(CHECKPOINT_BASE, True)

# Wyczyść cache
# spark.catalog.clearCache()

# print("Zasoby zostały wyczyszczone")
</VSCode.Cell>
```